In [ ]:
import numpy as np
from numba import njit
from random import randint
import time
import timeit

In [ ]:
@njit
def run():
    d = np.zeros((64,64), dtype=np.int8) #only 1 or 0
    U = np.zeros((64,64), dtype=np.int16) #can be negative
    V = np.zeros((64,64), dtype=np.int8) #only 1 or 0
    dU = np.zeros((64,64), dtype=np.int8)#can be negative
    
    for i in range(64):
        for j in range(i+1,64):
            dx = (i//8)-(j//8)
            dy = (i%8)-(j%8)
            if(((dx*dx)+(dy*dy))==5):
                d[i][j] = 1
                d[j][i] = 1
                V[i][j] = randint(0,1)
                V[j][i] = V[i][j]
    
    counter = 0
    while True:
        counter += 1
        
        for i in range(64):
            for j in range(64):
                if d[i][j] == 1:
#                     delta = 4
#                     for k in range(64): 
#                         #an optimization here might be to not do the sum each time, 
#                         #and instead, update the sum during the next step
# #                         delta -= ((d[i][k]*V[i][k])+(d[j][k]*V[j][k]))
#                         delta -= (V[i][k] + V[j][k])
                    dU[i][j] = 4 - (np.sum(V[i]) + np.sum(V[j]))
        
        U = np.add(U,dU,U)
        for i in range(64):
            for j in range(64):
#                 U[i][j] += dU[i][j]
                if U[i][j] > 3:
                    V[i][j] = 1
                elif U[i][j] < 0:
                    V[i][j] = 0
                else:
                    pass
        
        if counter % 1000 == 0:
            sum_dU = np.sum(np.absolute(dU))
            max_U = np.max(np.absolute(U))
            if max_U > 255: 
                return [(np.int64(x), np.int64(x)) for x in range(0)]            
            if counter > 200000:
                return [(np.int64(x), np.int64(x)) for x in range(0)]
            if sum_dU == 0:
                soln = [(np.int64(x), np.int64(x)) for x in range(0)]
                for i in range(64):
                    for j in range(64):
                        if V[i][j] == 1:
                            soln.append((i,j))
#                             print(i, j)
                return soln

In [ ]:
def get_sol():
    while True:
        soln = run()
        if len(soln) > 0:
            edges = [(min(edge),max(edge)) for edge in soln]
            edges = sorted(list(set(edges)), key=lambda x: x[0])
#             print(len(edges))
            sol = np.zeros(64, dtype=np.int64)
            sol[0]=1
            cr=0
            for q in range(1,64):
                for l in range(len(edges)):
                    if edges[l][0] == cr or edges[l][1] == cr:
                        oth = edges[l][0] if edges[l][1] == cr else edges[l][1]
                    if sol[oth]==0:
                        sol[oth]=q+1;
                        cr=oth;
                        break
            if 0 in sol:
                continue

            for i in range(8):
                for j in range(8):
                    print(str(sol[i*8+j]).ljust(2), end=' ')
                print()

            break

In [ ]:
times = []
for i in range (100):
    start = time.time()
    get_sol()
    taken = int(time.time()-start)
    print(str(taken) + " seconds taken.")
    times.append(taken)

In [ ]:
t = np.array(sorted(times))
print("average: {}".format(np.average(t)))
print("std dev: {}".format(np.std(t)))
print("median: {}".format(np.median(t)))
print("75th percentile: {}".format(t[74]))
i = 0
while t[i] <= 60 and i < len(t):
    i+=1
print("{} percent less than 1 min".format(((i+1)*100)/len(t)))

In [ ]:
t